# 🧹 Limpieza y Preparación de Datos - Superstore

Este notebook realiza la limpieza y transformación del dataset Superstore para su posterior análisis en Power BI.

## Contenido
1. Carga de datos
2. Exploración inicial
3. Limpieza de datos
4. Creación de nuevas variables
5. Exportación del dataset limpio

## 1. Importar librerías y cargar datos

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Cargar dataset
df = pd.read_csv('../data/Sample - Superstore.csv', encoding='latin-1')

print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")

## 2. Exploración inicial

In [ ]:
# Ver primeras filas
df.head()

In [ ]:
# Información del dataset
df.info()

In [ ]:
# Verificar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

In [ ]:
# Estadísticas descriptivas
df.describe()

In [ ]:
# Verificar duplicados
duplicados = df.duplicated().sum()
print(f"Filas duplicadas: {duplicados}")

## 3. Limpieza de datos

In [ ]:
# Eliminar columna Row ID (no aporta valor)
df = df.drop('Row ID', axis=1)

print("Columna 'Row ID' eliminada")

In [ ]:
# Convertir fechas a datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%m/%d/%Y')

print("Fechas convertidas a datetime")
print(f"Rango de fechas: {df['Order Date'].min()} a {df['Order Date'].max()}")

In [ ]:
# Verificar valores únicos en columnas categóricas
categoricas = ['Ship Mode', 'Segment', 'Category', 'Sub-Category', 'Region']

for col in categoricas:
    print(f"\n{col}: {df[col].nunique()} valores únicos")
    print(df[col].value_counts())

## 4. Creación de nuevas variables

In [ ]:
# Variables temporales
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Month_Name'] = df['Order Date'].dt.month_name()
df['Quarter'] = df['Order Date'].dt.quarter
df['Day_of_Week'] = df['Order Date'].dt.day_name()
df['Week_of_Year'] = df['Order Date'].dt.isocalendar().week

print("Variables temporales creadas: Year, Month, Month_Name, Quarter, Day_of_Week, Week_of_Year")

In [ ]:
# Días de envío
df['Shipping_Days'] = (df['Ship Date'] - df['Order Date']).dt.days

print(f"Días de envío - Promedio: {df['Shipping_Days'].mean():.1f}, Máximo: {df['Shipping_Days'].max()}")

In [ ]:
# Margen de ganancia
df['Profit_Margin'] = (df['Profit'] / df['Sales'] * 100).round(2)

# Reemplazar infinitos y NaN por 0
df['Profit_Margin'] = df['Profit_Margin'].replace([np.inf, -np.inf], 0).fillna(0)

print(f"Margen de ganancia - Promedio: {df['Profit_Margin'].mean():.2f}%")

In [ ]:
# Categoría de descuento
def categorize_discount(discount):
    if discount == 0:
        return 'Sin descuento'
    elif discount <= 0.1:
        return 'Bajo (1-10%)'
    elif discount <= 0.2:
        return 'Medio (11-20%)'
    else:
        return 'Alto (>20%)'

df['Discount_Category'] = df['Discount'].apply(categorize_discount)

print("Categorías de descuento:")
print(df['Discount_Category'].value_counts())

In [ ]:
# Indicador de ganancia/pérdida
df['Profit_Flag'] = df['Profit'].apply(lambda x: 'Ganancia' if x > 0 else 'Pérdida')

print("Distribución Ganancia/Pérdida:")
print(df['Profit_Flag'].value_counts())

## 5. Resumen del dataset limpio

In [ ]:
print("=" * 50)
print("RESUMEN DEL DATASET LIMPIO")
print("=" * 50)
print(f"Total de registros: {len(df):,}")
print(f"Total de columnas: {len(df.columns)}")
print(f"Período: {df['Year'].min()} - {df['Year'].max()}")
print(f"\nVentas totales: ${df['Sales'].sum():,.2f}")
print(f"Ganancias totales: ${df['Profit'].sum():,.2f}")
print(f"Margen promedio: {(df['Profit'].sum() / df['Sales'].sum() * 100):.2f}%")
print(f"\nÓrdenes únicas: {df['Order ID'].nunique():,}")
print(f"Clientes únicos: {df['Customer ID'].nunique():,}")
print(f"Productos únicos: {df['Product ID'].nunique():,}")

In [ ]:
# Ver columnas finales
print("Columnas del dataset limpio:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

## 6. Exportar dataset limpio

In [ ]:
# Exportar a CSV
df.to_csv('../data/superstore_clean.csv', index=False)

print("Dataset exportado exitosamente: ../data/superstore_clean.csv")

In [ ]:
# Verificar exportación
df_check = pd.read_csv('../data/superstore_clean.csv')
print(f"Verificación: {df_check.shape[0]} filas, {df_check.shape[1]} columnas")